In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import CubicSpline
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

In [ ]:
from fcmeans import FCM

In [ ]:
from scipy.stats import moment
from scipy.stats import skew

colors = ['r', 'g', 'b']
labels_str = ['Slug', 'Intermittent', 'Annular']
markers = ['o', '^', 's']  # circle, triangle, square

df = pd.read_csv('volume-average-rfile.csv')
df['time'] = df['time'] * 0.01

C_liquid = 1.113e-10 * 1.51 * (10*10)/10
C_gas = 1.113e-10 * 1.000494 * (10*10)/10

def convert_to_capacitance(void_fraction, C_l, C_g):
    return C_l - (void_fraction * (C_l - C_g))

df['capacitance'] = df['Volume fraction'].apply(lambda x: convert_to_capacitance(x, C_liquid, C_gas))

def create_intervals(data, interval_size):
    intervals = []
    stats = []

    for i in range(0, len(data) - interval_size + 1, interval_size):
        interval = data.iloc[i:i+interval_size]

        mean = interval['capacitance'].mean()
        variance = interval['capacitance'].var()
        skewness = skew(interval['capacitance'])
        kurtosis = moment(interval['capacitance'], moment=4)

        stats.append([mean, variance, kurtosis])

    return np.array(stats)

# After reading and initial processing of the data
# Add this code before the interpolation section

# Split data into flow regimes and filter annular data
def categorize_flow(time):
    if time <= 0.85:
        return 'Slug'
    elif time <= 1.5:
        return 'Intermittent'
    else:
        return 'Annular'

# Split data into flow regimes
df['flow_regime'] = df['time'].apply(categorize_flow)
slug_data = df[df['flow_regime'] == 'Slug']
intermittent_data = df[df['flow_regime'] == 'Intermittent']

# Filter annular data up to specific time (e.g., time <= 5.0)
max_annular_time = 3.5  # You can adjust this value
annular_data = df[(df['flow_regime'] == 'Annular') & (df['time'] <= max_annular_time)]

# Interpolate slug and intermittent data
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, Matern

def interpolate_selected_data(original_df, n_points_between):
    # Try a more flexible kernel
    kernel = 1.0 * RBF(length_scale=0.2) + WhiteKernel(noise_level=0.05)
    gpr = GaussianProcessRegressor(kernel=kernel, optimizer='fmin_l_bfgs_b', random_state=0)

    X = original_df['time'].values.reshape(-1, 1)
    y = original_df['Volume fraction'].values
    gpr.fit(X, y)

    # Create new time points
    new_times = []
    for i in range(len(X)-1):
        t_start = X[i][0]
        t_end = X[i+1][0]
        t_between = np.linspace(t_start, t_end, n_points_between + 2)[1:-1]
        new_times.extend(t_between)
    all_times = np.sort(np.concatenate([X.flatten(), np.array(new_times)]))

    predictions, std = gpr.predict(all_times.reshape(-1, 1), return_std=True)

    df_pred = pd.DataFrame({
        'time': all_times,
        'Volume fraction': predictions,
        'uncertainty': std
    })
    df_pred['capacitance'] = df_pred['Volume fraction'].apply(
        lambda x: convert_to_capacitance(x, C_liquid, C_gas)
    )
    return df_pred


# Interpolate slug and intermittent regions
slug_interpolated = interpolate_selected_data(slug_data, 2) # 9
intermittent_interpolated = interpolate_selected_data(intermittent_data, 2) # 12

# Add capacitance to annular data if not already present
if 'capacitance' not in annular_data.columns:
    annular_data['capacitance'] = annular_data['Volume fraction'].apply(
        lambda x: convert_to_capacitance(x, C_liquid, C_gas)
    )

df_interpolated = pd.concat([
    slug_interpolated,
    intermittent_interpolated,
    annular_data[['time', 'Volume fraction', 'capacitance', 'flow_regime']]
]).sort_values('time').reset_index(drop=True)

df_combined = pd.concat([
    df[['time', 'Volume fraction', 'capacitance', 'flow_regime']],
    df_interpolated[['time', 'Volume fraction', 'capacitance', 'flow_regime']]
]).sort_values('time').reset_index(drop=True)

# Create features from combined dataset
features_combined = create_intervals(df_combined, 20)

# Scale features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_combined)

# Fit clustering model on combined features
fcm = FCM(n_clusters=3)
fcm.fit(features_scaled)

def relabel_clusters(labels, features_scaled):
    # Get mean capacitance (first feature) for each cluster
    cluster_means = []
    for i in range(3):
        cluster_points = features_scaled[labels == i]
        mean_capacitance = np.mean(cluster_points[:, 0])  # First feature is mean capacitance
        cluster_means.append((i, mean_capacitance))

    # Sort clusters by mean capacitance
    # Slug has lowest capacitance, Annular has highest
    sorted_clusters = sorted(cluster_means, key=lambda x: x[1])

    # Create mapping from original labels to correct order
    mapping = {
        sorted_clusters[0][0]: 2,  # Slug
        sorted_clusters[1][0]: 1,  # Intermittent
        sorted_clusters[2][0]: 0   # Annular
    }

    # Relabel the clusters
    new_labels = np.array([mapping[label] for label in labels])
    return new_labels

# After FCM clustering, relabel the clusters
labels = relabel_clusters(fcm.u.argmax(axis=1), features_scaled)

# Define common style parameters
plt.style.use('default')

# Define colors and markers
colors = ['red', 'green', 'blue']  # Standard RGB colors
labels_str = ['Slug', 'Intermittent', 'Annular']
markers = ['o', '^', 's']  # circle, triangle, square

# Common font sizes
TITLE_SIZE = 16
AXIS_LABEL_SIZE = 14
LEGEND_SIZE = 12

# 3D Plot (fig1)
fig1 = plt.figure(figsize=(10, 8))
ax1 = fig1.add_subplot(111, projection='3d')

for i in range(3):
    cluster_points = features_scaled[labels == i]
    ax1.scatter(cluster_points[:, 0],
              cluster_points[:, 1],
              cluster_points[:, 2],
              marker=markers[i],
              facecolors='none',
              edgecolors=colors[i],
              s=100,
              linewidth=1.5,
              label=labels_str[i])

ax1.set_xlabel('Normalized Mean Capacitance', fontsize=AXIS_LABEL_SIZE)
ax1.set_ylabel('Normalized Variance', fontsize=AXIS_LABEL_SIZE)
ax1.set_zlabel('Normalized Kurtosis', fontsize=AXIS_LABEL_SIZE)
ax1.legend(fontsize=LEGEND_SIZE)
ax1.set_title('3D View of Flow Patterns', fontsize=TITLE_SIZE)
plt.tight_layout()

# Mean vs Variance Plot (fig2)
fig2 = plt.figure(figsize=(10, 8))
ax2 = fig2.add_subplot(111)

for i in range(3):
    cluster_points = features_scaled[labels == i]
    ax2.scatter(cluster_points[:, 0],
               cluster_points[:, 1],
               marker=markers[i],
               facecolors='none',
               edgecolors=colors[i],
               s=100,
               linewidth=1.5,
               label=labels_str[i])

ax2.set_xlabel('Normalized Mean Capacitance', fontsize=AXIS_LABEL_SIZE)
ax2.set_ylabel('Normalized Variance', fontsize=AXIS_LABEL_SIZE)
ax2.legend(fontsize=LEGEND_SIZE)
ax2.set_title('Mean vs Variance View of Flow Patterns', fontsize=TITLE_SIZE)
ax2.grid(True, alpha=0.3)
plt.tight_layout()

# Variance vs Kurtosis Plot (fig3)
fig3 = plt.figure(figsize=(10, 8))
ax3 = fig3.add_subplot(111)

for i in range(3):
    cluster_points = features_scaled[labels == i]
    ax3.scatter(cluster_points[:, 1],
               cluster_points[:, 2],
               marker=markers[i],
               facecolors='none',
               edgecolors=colors[i],
               s=100,
               linewidth=1.5,
               label=labels_str[i])

ax3.set_xlabel('Normalized Variance', fontsize=AXIS_LABEL_SIZE)
ax3.set_ylabel('Normalized Kurtosis', fontsize=AXIS_LABEL_SIZE)
ax3.legend(fontsize=LEGEND_SIZE)
ax3.set_title('Variance vs Kurtosis View of Flow Patterns', fontsize=TITLE_SIZE)
ax3.grid(True, alpha=0.3)
plt.tight_layout()

# Save figures with black background
fig1.savefig('3d_plot.png', bbox_inches='tight', pad_inches=0.5, dpi=300,
              edgecolor='none')
fig2.savefig('mean_variance_plot.png', bbox_inches='tight', pad_inches=0.5, dpi=300,
              edgecolor='none')
fig3.savefig('variance_kurtosis_plot.png', bbox_inches='tight', pad_inches=0.5, dpi=300,
             edgecolor='none')

plt.show()

def categorize_flow(time):
    if time <= 0.85:
        return 'Slug'
    elif time <= 1.5:
        return 'Intermittent'
    else:
        return 'Annular'

# Apply categorization to the time data
df['flow_regime'] = df['time'].apply(categorize_flow)


# Reset matplotlib style to default (light mode)
plt.style.use('default')

# Define larger font sizes
TITLE_SIZE = 20
LABEL_SIZE = 16
LEGEND_SIZE = 14
PERCENT_SIZE = 14

# Define consistent color mapping
flow_colors = {
    'Annular': 'lightblue',
    'Intermittent': 'lightgreen',
    'Slug': 'lightcoral'
}

# Modified function for labeling with larger font size
def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        return f'{pct:.1f}%\n({val:d})'
    return lambda pct: f'$\mathbf{{{pct:.1f}\%}}$\n({int(round(pct*sum(values)/100.0)):d})'

# Create and save Original Data Pie Chart
plt.figure(figsize=(12, 10))  # Increased figure size
original_regime_counts = df['flow_regime'].value_counts()

# Sort the counts to maintain consistent order
ordered_regimes = ['Slug', 'Intermittent', 'Annular']
ordered_counts = [original_regime_counts.get(regime, 0) for regime in ordered_regimes]
ordered_colors = [flow_colors[regime] for regime in ordered_regimes]

plt.pie(ordered_counts,
        labels=ordered_regimes,
        autopct=make_autopct(ordered_counts),
        colors=ordered_colors,
        explode=(0.05, 0.05, 0.05),
        textprops={'fontsize': LABEL_SIZE})  # Increased label font size
plt.title('Original Distribution of Flow Regimes', fontsize=TITLE_SIZE, pad=20, fontweight='bold')

# Add legend with consistent order and larger font
plt.legend(ordered_regimes,
          title="Flow Regimes",
          title_fontsize=LEGEND_SIZE,  # Increased legend title font size
          fontsize=LEGEND_SIZE-2,      # Increased legend text font size
          loc="center left",
          bbox_to_anchor=(1, 0, 0.5, 1))

plt.tight_layout()
plt.savefig('original_flow_regime_pie.png',
            bbox_inches='tight',
            dpi=300)
plt.close()

# Create and save Interpolated Data Pie Chart with same formatting
plt.figure(figsize=(12, 10))
df_interpolated['flow_regime'] = df_interpolated['time'].apply(categorize_flow)
interpolated_regime_counts = df_interpolated['flow_regime'].value_counts()

ordered_counts_interp = [interpolated_regime_counts.get(regime, 0) for regime in ordered_regimes]

plt.pie(ordered_counts_interp,
        labels=ordered_regimes,
        autopct=make_autopct(ordered_counts_interp),
        colors=ordered_colors,
        explode=(0.05, 0.05, 0.05),
        textprops={'fontsize': LABEL_SIZE})
plt.title('Interpolated Distribution of Flow Regimes', fontsize=TITLE_SIZE, pad=20, fontweight='bold')

plt.legend(ordered_regimes,
          title="Flow Regimes",
          title_fontsize=LEGEND_SIZE,
          fontsize=LEGEND_SIZE-2,
          loc="center left",
          bbox_to_anchor=(1, 0, 0.5, 1))

plt.tight_layout()
plt.savefig('interpolated_flow_regime_pie.png',
            bbox_inches='tight',
            dpi=300)
plt.close()

# Print exact counts
print("\nOriginal Data Points in each regime:")
for regime, count in zip(ordered_regimes, ordered_counts):
    print(f"{regime}: {count} points")

print("\nInterpolated Data Points in each regime:")
for regime, count in zip(ordered_regimes, ordered_counts_interp):
    print(f"{regime}: {count} points")